In [1]:
# Copyright (c) 2020 The University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


"""
Modifications: Ported the model to Izhikevich's Conductance neurons
Developed as a part of the undergraduate project by Ishita Mediratta under
the guidance of Dr. Basabdatta Sen-Bhattacharya

The model is given Poisson input in the beta range with parameters tuned in
a way that they addhere to the plausible irregularity and synchrony values

Original Implementation:

Version uploaded on ModelDB October 2017.
Author:
Basabdatta Sen Bhattacharya, APT group, School of Computer Science,
University of Manchester, 2017.

If you are using the code,
please cite the original work on the model - details are:

B. Sen-Bhattacharya, T. Serrano-Gotarredona, L. Balassa, A. Bhattacharya,
A.B. Stokes, A. Rowley, I. Sugiarto, S.B. Furber,
"A spiking neural network model of the Lateral Geniculate Nucleus on the
SpiNNaker machine", Frontiers in Neuroscience, vol. 11 (454), 2017.

Free online access:
https://journal.frontiersin.org/article/10.3389/fnins.2017.00454/abstract
"""

import pyNN.spiNNaker as p
import numpy as np
import math
from pyNN.random import RandomDistribution, NumpyRNG

# for plotting
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt

# pylint: disable=pointless-string-statement


def get_mean_rate(numCells, population):
    firing_rate = []      # format = < neuron_id, rate (spikes/ms) >

    for index in range(0, numCells):
        rate = len(population.segments[0].spiketrains[index])/TotalDuration
        firing_rate.append(rate)

    return sum(firing_rate)/len(firing_rate)


def calc_irregularity(segment):
    irregularity = 0
    isi_array = []
    for i in range(len(segment.spiketrains)):
        if len(segment.spiketrains[i]) > 2:
            isi_array.append([])
            for j in range(len(segment.spiketrains[i])-1):
                isi_array[-1].append(
                    segment.spiketrains[i][j+1]-segment.spiketrains[i][j])
    for i in range(len(isi_array)):
        mean = np.mean(isi_array[i])
        sd = np.std(isi_array[i])
        cv = sd / mean
        irregularity += cv
    irregularity = irregularity / len(segment.spiketrains)
    return irregularity


def print_irregularity():
    print("TCR irregularity: ", calc_irregularity(TCR_spikes.segments[0]))
    print("IN irregularity: ", calc_irregularity(IN_spikes.segments[0]))
    print("TRN irregularity: ", calc_irregularity(TRN_spikes.segments[0]))


def calc_synchrony(segment):
    spike_counts = np.zeros(int(TotalDuration/2.0), dtype=int)
    for i in range(len(segment.spiketrains)):
        for j in range(len(segment.spiketrains[i])):
            index = math.floor(segment.spiketrains[i][j] / 2.0)
            spike_counts[index] += 1
    mean = np.mean(spike_counts)
    var = np.std(spike_counts) * np.std(spike_counts)
    synchrony = var / mean
    return synchrony


def print_synchrony():
    print("TCR synchrony: ", calc_synchrony(TCR_spikes.segments[0]))
    print("IN synchrony: ", calc_synchrony(IN_spikes.segments[0]))
    print("TRN synchrony: ", calc_synchrony(TRN_spikes.segments[0]))


""" Initialising Time and Frequency parameters """

# total duration of simulation
TotalDuration = int(1000)

# this is in ms.
Duration_Inp = int(1000)

# 50 ms at both start and end are disregarded to avoid transients
Start_Inp = int(0)
End_Inp = int(Start_Inp + Duration_Inp)

Rate_Inp = int(22)
Inp_isi = int(1000 / Rate_Inp)

""" Initialising Model connectivity parameters """

intra_pop_delay = RandomDistribution('uniform', (1, 3))
intra_nucleus_delay = RandomDistribution('uniform', (1, 3))
inter_nucleus_delay = RandomDistribution('uniform', (1, 3))
inter_pop_delay = RandomDistribution('uniform', (1, 3))
input_delay = inter_pop_delay

# # input_delay is the delay of the spike source hitting the neuronal pops
# # inter_pop_delay is the delay of spike communication between the different
# # populations of the model

# probabilities
p_trn2trn = 0.15
p_in2tcr = 0.1545  # 0.232
p_in2in = 0.236
p_tcr2trn = 0.35
p_trn2tcr = 0.1545  # 0.07
p_ret2tcr = 0.07
p_ret2in = 0.47

# weights
w_trn2trn = 1.0  # 0.06 # 1
w_in2tcr = 0.1  # 2
w_in2in = 0.35  # 2
w_tcr2trn = 0.115  # 0.01 # 2 # 0.2 and 0.3 give best value -> 0.25
w_trn2tcr = 0.03  # 2
w_ret2tcr = 0.275  # 0.35 # 0.1 # 1
w_ret2in = 0.275  # 0.35 # 0.1 # 1

""" Initialising Izhikevich spiking neuron model parameters.
We have used the conductance-based model here. """

# Tonic mode parameters
tcr_a_tonic = 0.02
tcr_b_tonic = 0.2
tcr_c_tonic = -65.0
tcr_d_tonic = 6.0
tcr_v_init_tonic = RandomDistribution('uniform', (-63.0, -67.0),
                                      rng=NumpyRNG(seed=85520))  # -65.0

in_a_tonic = 0.1
in_b_tonic = 0.2
in_c_tonic = -65.0
in_d_tonic = 6.0
in_v_init_tonic = RandomDistribution('uniform', (-68.0, -72.0),
                                     rng=NumpyRNG(seed=85521))  # -70.0

trn_a_tonic = 0.02
trn_b_tonic = 0.2
trn_c_tonic = -65.0
trn_d_tonic = 6.0
trn_v_init_tonic = RandomDistribution('uniform', (-73.0, -77.0),
                                      rng=NumpyRNG(seed=85522))  # -75.0

tcr_a = tcr_a_tonic
tcr_b = tcr_b_tonic
tcr_c = tcr_c_tonic
tcr_d = tcr_d_tonic
tcr_v_init = tcr_v_init_tonic

in_a = in_a_tonic
in_b = in_b_tonic
in_c = in_c_tonic
in_d = in_d_tonic
in_v_init = in_v_init_tonic

trn_a = trn_a_tonic
trn_b = trn_b_tonic
trn_c = trn_c_tonic
trn_d = trn_d_tonic
trn_v_init = trn_v_init_tonic

# tcr_b * tcr_v_init
tcr_u_init = RandomDistribution('uniform', (-15.0, -11.0),
                                rng=NumpyRNG(seed=85522))  # -13.0
# in_b * in_v_init
in_u_init = RandomDistribution('uniform', (-16.0, -12.0),
                               rng=NumpyRNG(seed=85522))  # -14.0
# trn_b * trn_v_init
trn_u_init = RandomDistribution('uniform', (-17.0, -13.0),
                                rng=NumpyRNG(seed=85522))  # -15.0

# a constant DC bias current; this is used here for testing the RS and FS
# characteristics of IZK neurons
current_Pulse = RandomDistribution('poisson', lambda_=3.0,
                                   rng=NumpyRNG(seed=85524))  # 5

# excitatory input time constant
tau_ex = 6.0

# inhibitory input time constant
tau_inh = 4.0

# reversal potentials
e_rev_ex = 0.0
e_rev_inh = -80.0

""" Starting the SpiNNaker Simulator """
p.setup(timestep=0.1)
# set number of neurons per core to 50, for the spike source to avoid clogging
# p.set_number_of_neurons_per_core(p.SpikeSourceArray, 50)

""" Defining each cell type as dictionary """

# THALAMOCORTICAL RELAY CELLS (TCR)
TCR_cell_params = {'a': tcr_a_tonic, 'b': tcr_b, 'c': tcr_c, 'd': tcr_d,
                   'tau_syn_E': tau_ex, 'tau_syn_I': tau_inh,
                   'i_offset': current_Pulse, 'e_rev_E': e_rev_ex,
                   'e_rev_I': e_rev_inh
                   }

TCR_initial_values = {'v': tcr_v_init, 'u': tcr_u_init}

# THALAMIC INTERNEURONS (IN)
IN_cell_params = {'a': in_a, 'b': in_b, 'c': in_c, 'd': in_d,
                  'tau_syn_E': tau_ex, 'tau_syn_I': tau_inh,
                  'i_offset': current_Pulse, 'e_rev_E': e_rev_ex,
                  'e_rev_I': e_rev_inh
                  }

IN_initial_values = {'v': in_v_init, 'u': in_u_init}

# THALAMIC RETICULAR NUCLEUS (TRN)
TRN_cell_params = {'a': trn_a, 'b': trn_b, 'c': trn_c, 'd': trn_d,
                   'tau_syn_E': tau_ex, 'tau_syn_I': tau_inh,
                   'i_offset': current_Pulse, 'e_rev_E': e_rev_ex,
                   'e_rev_I': e_rev_inh
                   }

TRN_initial_values = {'v': trn_v_init, 'u': trn_u_init}

""" Creating populations of each cell type """
scale_fact = 10
NumCellsTCR = 8*scale_fact
NumCellsIN = 2*scale_fact
NumCellsTRN = 4*scale_fact
TCR_pop = p.Population(
    NumCellsTCR, p.extra_models.Izhikevich_cond, TCR_cell_params,
    label='TCR_pop', initial_values=TCR_initial_values, seed=85520)
IN_pop = p.Population(
    NumCellsIN, p.extra_models.Izhikevich_cond, IN_cell_params,
    label='IN_pop', initial_values=IN_initial_values, seed=85521)
TRN_pop = p.Population(
    NumCellsTRN, p.extra_models.Izhikevich_cond, TRN_cell_params,
    label='TRN_pop', initial_values=TRN_initial_values, seed=85522)

""" Poisson input for TCR """
spike_source_TCR = p.Population(
    NumCellsTCR, p.SpikeSourcePoisson(rate=10, start=Start_Inp,
                                      duration=Duration_Inp),
    label='spike_source_TCR', seed=85523)

""" Poisson input for IN """
spike_source_IN = p.Population(
    NumCellsIN, p.SpikeSourcePoisson(rate=10, start=Start_Inp,
                                     duration=Duration_Inp),
    label='spike_source_IN', seed=85524)

""" Poisson Source to TCR population projections """
Proj0 = p.Projection(
    spike_source_TCR, TCR_pop, p.OneToOneConnector(),
    p.StaticSynapse(weight=w_ret2tcr, delay=input_delay),
    receptor_type='excitatory')


""" Poisson Source2IN """
Proj1 = p.Projection(
    spike_source_IN, IN_pop, p.OneToOneConnector(),
    p.StaticSynapse(weight=w_ret2in, delay=input_delay),
    receptor_type='excitatory')


""" TCR2TRN """
Proj2 = p.Projection(
    TCR_pop, TRN_pop, p.FixedProbabilityConnector(p_connect=p_tcr2trn),
    p.StaticSynapse(weight=w_tcr2trn, delay=inter_nucleus_delay),
    receptor_type='excitatory')


""" TRN2TCR """
Proj3 = p.Projection(
    TRN_pop, TCR_pop, p.FixedProbabilityConnector(p_connect=p_trn2tcr),
    p.StaticSynapse(weight=w_trn2tcr, delay=inter_nucleus_delay),
    receptor_type='inhibitory')


""" TRN2TRN """
Proj4 = p.Projection(
    TRN_pop, TRN_pop, p.FixedProbabilityConnector(p_connect=p_trn2trn),
    p.StaticSynapse(weight=w_trn2trn, delay=intra_pop_delay),
    receptor_type='inhibitory')


""" IN2TCR """
Proj5 = p.Projection(
    IN_pop, TCR_pop, p.FixedProbabilityConnector(p_connect=p_in2tcr),
    p.StaticSynapse(weight=w_in2tcr, delay=intra_nucleus_delay),
    receptor_type='inhibitory')


""" IN2IN """
Proj6 = p.Projection(
    IN_pop, IN_pop, p.FixedProbabilityConnector(p_connect=p_in2in),
    p.StaticSynapse(weight=w_in2in, delay=intra_pop_delay),
    receptor_type='inhibitory')

""" Recording simulation data"""

# recording the spikes and voltage
spike_source_TCR.record("spikes")
spike_source_IN.record("spikes")
# spike_source_periodic_TCR.record("spikes")
# spike_source_periodic_IN.record("spikes")
TCR_pop.record(("spikes", "v", "gsyn_exc", "gsyn_inh"))
IN_pop.record(("spikes", "v", "gsyn_exc", "gsyn_inh"))
TRN_pop.record(("spikes", "v", "gsyn_exc", "gsyn_inh"))

p.run(TotalDuration)

""" On simulation completion, extract the data off the spinnaker machine
memory """

# extracting the spike time data
# spikesourcepattern_TCR = spike_source_periodic_TCR.get_data("spikes")
# spikesourcepattern_IN = spike_source_periodic_IN.get_data("spikes")
spikesourcepattern_TCR = spike_source_TCR.get_data("spikes")
spikesourcepattern_IN = spike_source_IN.get_data("spikes")
TCR_spikes = TCR_pop.get_data("spikes")
IN_spikes = IN_pop.get_data("spikes")
TRN_spikes = TRN_pop.get_data("spikes")

# extracting the membrane potential data (in millivolts)
TCR_membrane_volt = TCR_pop.get_data("v")
IN_membrane_volt = IN_pop.get_data("v")
TRN_membrane_volt = TRN_pop.get_data("v")

# print TCR_membrane_volt.segments[0].analogsignals
TCR_gsyn_e = TCR_pop.get_data("gsyn_exc")
IN_gsyn_e = IN_pop.get_data("gsyn_exc")
TRN_gsyn_e = TRN_pop.get_data("gsyn_exc")

TCR_gsyn_i = TCR_pop.get_data("gsyn_inh")
IN_gsyn_i = IN_pop.get_data("gsyn_inh")
TRN_gsyn_i = TRN_pop.get_data("gsyn_inh")

print_irregularity()
print_synchrony()
print(get_mean_rate(NumCellsTCR, TCR_spikes)*1000)
print(get_mean_rate(NumCellsIN, IN_spikes)*1000)
print(get_mean_rate(NumCellsTRN, TRN_spikes)*1000)

""" Plotting """

Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(TCR_spikes.segments[0].spiketrains, xlabel="Time/ms",
          xticks=True, ylabel="TCR Spikes Plots for TotalDuration",
          yticks=True, markersize=0.5, xlim=(1, TotalDuration), color='red'),
    Panel(IN_spikes.segments[0].spiketrains, xlabel="Time/ms",
          xticks=True, ylabel="IN Spikes Plots for TotalDuration",
          yticks=True, markersize=0.5, xlim=(1, TotalDuration), color='red'),
    Panel(TRN_spikes.segments[0].spiketrains, xlabel="Time/ms",
          xticks=True, ylabel="TRN Spikes Plots for TotalDuration",
          yticks=True, markersize=0.5, xlim=(1, TotalDuration), color='red'),
    Panel(TCR_membrane_volt.segments[0].filter(name="v")[0], xlabel="Time/ms",
          xticks=True, ylabel="TCR membrane voltage",
          yticks=True, markersize=0.5, xlim=(100, 400), legend=False),
    Panel(IN_membrane_volt.segments[0].filter(name="v")[0], xlabel="Time/ms",
          xticks=True, ylabel="IN membrane voltage",
          yticks=True, markersize=0.5, xlim=(100, 400), legend=False),
    Panel(TRN_membrane_volt.segments[0].filter(name="v")[0], xlabel="Time/ms",
          xticks=True, ylabel="TRN membrane voltage",
          yticks=True, markersize=0.5, xlim=(100, 400), legend=False),
    title="Effect of I_DC on periodic input, with Izhikevich_cond neurons",
    annotations="Simulated with {}".format(p.name())
)
# plt.savefig("Effect of I_DC on periodic input.png")
plt.show()

p.end()

2024-04-25 11:49:43 INFO: Read configs files: /home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg, /home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg, /home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2024-04-25 11:49:43 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/model_binaries
2024-04-25 11:49:43 WARNING: A timestep was entered that has forced spinnaker to automatically slow the simulation down from real time by a factor of 10.
2024-04-25 11:49:43 INFO: Setting hardware timestep as 1000 microseconds based on simulation time step of 100 and timescale factor of 10
2024-04-25 11:49:43 INFO: Starting execution process
2024-04-25 11:49

['/home/bbpnrsoa/sPyNNakerGit/SpiNNUtils/spinn_utilities/spinn_utilities.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMachine/spinn_machine/spinn_machine.cfg', '/home/bbpnrsoa/sPyNNakerGit/PACMAN/pacman/pacman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNMan/spinnman/spinnman.cfg', '/home/bbpnrsoa/sPyNNakerGit/SpiNNFrontEndCommon/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNakerGit/sPyNNaker/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


2024-04-25 11:49:44 INFO: Spynnaker splitter selector took 0:00:00.072055 
Adding delay extensions as required
|0%                          50%                         100%|
2024-04-25 11:49:44 INFO: DelaySupportAdder took 0:00:00.080016 
Partitioning Graph
|0%                          50%                         100%|
2024-04-25 11:49:44 INFO: Splitter partitioner took 0:00:00.072651 
2024-04-25 11:49:44 INFO: 0.02 Boards Required for 1 chips
2024-04-25 11:49:44 INFO: Requesting job with 1 boards
Created spalloc job 439170
2024-04-25 11:49:44 INFO: Created spalloc job 439170
Job has been queued by the spalloc server.
2024-04-25 11:49:44 INFO: Job has been queued by the spalloc server.
Waiting for board power commands to complete.
2024-04-25 11:49:48 INFO: Waiting for board power commands to complete.
2024-04-25 11:49:57 INFO: SpallocAllocator took 0:00:13.071839 
2024-04-25 11:49:57 INFO: Creating transceiver for 10.11.238.185
2024-04-25 11:49:57 INFO: Working out if machine is booted

TCR irregularity:  0.8270500641812415
IN irregularity:  0.8585827081037454
TRN irregularity:  0.7412603547664613
TCR synchrony:  1.0895760549558393
IN synchrony:  0.9994871794871794
TRN synchrony:  2.4053809874723653
12.737500000000008
19.500000000000004
33.925


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
def extract_signal(block):
    # Extract the single AnalogSignal from the segment
    signal = block.segments[0].analogsignals[0]
    return np.array(signal)  # Convert to a NumPy array for easier manipulation

# Extract excitatory conductance data
TCR_gsyn_e_array = extract_signal(TCR_gsyn_e)
IN_gsyn_e_array = extract_signal(IN_gsyn_e)
TRN_gsyn_e_array = extract_signal(TRN_gsyn_e)

# Similar blocks for inhibitory data
TCR_gsyn_i_array = extract_signal(TCR_gsyn_i)
IN_gsyn_i_array = extract_signal(IN_gsyn_i)
TRN_gsyn_i_array = extract_signal(TRN_gsyn_i)

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def combined_lfp_from_absolute_conductances(gsyn_e_tcr, gsyn_i_tcr, gsyn_e_in, gsyn_i_in, gsyn_e_trn, gsyn_i_trn):
    # Sum absolute values of conductances across all neurons in each population and then across all populations
    total_lfp = (np.sum(np.abs(gsyn_e_tcr) + np.abs(gsyn_i_tcr), axis=1) +
                 np.sum(np.abs(gsyn_e_in) + np.abs(gsyn_i_in), axis=1) +
                 np.sum(np.abs(gsyn_e_trn) + np.abs(gsyn_i_trn), axis=1))
    return total_lfp

# Generate the time array and plot LFP
time_array = np.linspace(0, 1000, num=10000)  # Total time in ms
combined_lfp = combined_lfp_from_absolute_conductances(
    TCR_gsyn_e_array, TCR_gsyn_i_array,
    IN_gsyn_e_array, IN_gsyn_i_array,
    TRN_gsyn_e_array, TRN_gsyn_i_array
)

def plot_lfp(time_array, lfp_data, title):
    plt.figure(figsize=(12, 6))
    plt.plot(time_array, lfp_data, label='Combined LFP from All Populations (Absolute Conductances)')
    plt.title(title)
    plt.xlabel('Time (ms)')
    plt.ylabel('LFP (arbitrary units)')
    plt.legend()
    plt.show()

plot_lfp(time_array, combined_lfp, 'Combined LFP from Absolute Sum of Conductances')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming the get_data('v') method returns a Block that contains the AnalogSignal data
tcr_v = TCR_membrane_volt.segments[0].analogsignals[0]
in_v = IN_membrane_volt.segments[0].analogsignals[0]
trn_v = TRN_membrane_volt.segments[0].analogsignals[0]


# Function to calculate and plot the LFP from average membrane potentials of multiple neuron populations
def plot_average_lfp(v_tcr, v_in, v_trn, times, title, ax):
    # Convert to NumPy arrays if not already (depending on your framework)
    v_tcr_array = np.array(v_tcr)
    v_in_array = np.array(v_in)
    v_trn_array = np.array(v_trn)

    # Exclude the first 100 ms (1000 time points at 10,000 Hz)
    slice_idx = 1000
    v_tcr_sliced = v_tcr_array[slice_idx:, :]
    v_in_sliced = v_in_array[slice_idx:, :]
    v_trn_sliced = v_trn_array[slice_idx:, :]
    times_sliced = times[slice_idx:]

    # Calculate average membrane potential across all neurons in each population
    avg_v_tcr = np.mean(v_tcr_sliced, axis=1)
    avg_v_in = np.mean(v_in_sliced, axis=1)
    avg_v_trn = np.mean(v_trn_sliced, axis=1)

    # Combine averages from all populations, you might average them or sum them based on your definition of LFP
    combined_avg_lfp = (avg_v_tcr + avg_v_in + avg_v_trn) / 3  # Averaging approach

    # Plotting
    ax.plot(times_sliced, combined_avg_lfp, alpha=0.5)  # Partial transparency
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Time (ms)', fontsize=14)
    ax.set_ylabel('Average Membrane Potential (mV)', fontsize=14)
    ax.legend(['Average LFP'], loc='upper right')

# Assuming you already have the membrane potential arrays ready for TCR, IN, TRN populations
tcr_times = np.linspace(0, 1000, num=10000)  # Time vector from 0 to 1000 ms with 10,000 points

# Create subplot
fig, ax = plt.subplots(figsize=(10, 5))

# Extract analog signals and convert to numpy arrays if needed
tcr_v = np.array(TCR_membrane_volt.segments[0].analogsignals[0])
in_v = np.array(IN_membrane_volt.segments[0].analogsignals[0])
trn_v = np.array(TRN_membrane_volt.segments[0].analogsignals[0])

# Plot average LFP from TCR, IN, TRN neurons
plot_average_lfp(tcr_v, in_v, trn_v, tcr_times, 'Average LFP from TCR, IN, TRN Neurons', ax)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Example on how to extract membrane potentials assuming data is stored in a structured way
# This is hypothetical and depends on your specific data structure

# Assuming 'TCR_membrane_volt', 'IN_membrane_volt', 'TRN_membrane_volt' are PyNN Block objects containing membrane potential data
v_tcr = np.array(TCR_membrane_volt.segments[0].analogsignals[0])
v_in = np.array(IN_membrane_volt.segments[0].analogsignals[0])
v_trn = np.array(TRN_membrane_volt.segments[0].analogsignals[0])

# Verify the extraction
print(v_tcr.shape, v_in.shape, v_trn.shape)  # This should output the shape of the arrays

(10000, 80) (10000, 20) (10000, 40)


In [15]:
# Assume v_tcr, v_in, v_trn are your arrays containing membrane potentials for each population
# These arrays should be NumPy arrays with dimensions [time, neurons]
# Calculate the mean across neurons and then average these means across populations
v_tcr_mean = np.mean(v_tcr, axis=1) if v_tcr.size else np.zeros_like(time_array)
v_in_mean = np.mean(v_in, axis=1) if v_in.size else np.zeros_like(time_array)
v_trn_mean = np.mean(v_trn, axis=1) if v_trn.size else np.zeros_like(time_array)

combined_avg_lfp = (v_tcr_mean + v_in_mean + v_trn_mean) / 3

In [16]:
import numpy as np
import matplotlib.pyplot as plt

def perform_fft_and_plot_psd(lfp_data, sampling_rate, title):
    n = len(lfp_data)
    dt = 1 / sampling_rate
    fft_result = np.fft.fft(lfp_data)
    frequencies = np.fft.fftfreq(n, dt)
    
    # Only take the positive part of the spectrum
    positive_indices = frequencies > 0
    frequencies = frequencies[positive_indices]
    fft_result = fft_result[positive_indices]

    # PSD calculation
    psd = np.abs(fft_result) ** 2 / (n * sampling_rate)

    plt.figure(figsize=(10, 5))
    plt.plot(frequencies, psd)
    plt.title('PSD of ' + title)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD [V²/Hz]')
    plt.xlim(0, 100)  # Focus up to 100 Hz
    plt.grid(True)
    plt.show()

# Example parameters and usage
sampling_rate = 10000  # This should match the actual sampling rate used in your data acquisition or simulation

# Assuming `combined_lfp` is already calculated as the absolute summed LFP
perform_fft_and_plot_psd(combined_lfp, sampling_rate, 'Combined LFP from Absolute Conductances')

# Assuming `combined_avg_lfp` is calculated as the average of the membrane potentials
perform_fft_and_plot_psd(combined_avg_lfp, sampling_rate, 'Average Membrane Potential LFP')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
from scipy.signal import gaussian, convolve

def gaussian_kernel(size, sigma):
    """ Generate a Gaussian kernel, used for smoothing the spike train. """
    kernel = gaussian(size, std=sigma, sym=True)
    kernel /= kernel.sum()  # Normalize the kernel to ensure it sums to 1
    return kernel

In [23]:
def convert_spikes_to_continuous(spike_times, total_time, dt, kernel):
    """ Convert discrete spike times to a continuous signal by convolution with a Gaussian kernel. """
    # Create a timeline with zero amplitude
    time_array = np.linspace(0, total_time, int(total_time / dt) + 1)
    continuous_signal = np.zeros_like(time_array)
    
    # Convert spike times to indices
    indices = np.floor(spike_times / dt).astype(int)
    continuous_signal[indices] += 1  # Increment the bins where spikes occur
    
    # Convolve the spike counts with the Gaussian kernel
    smoothed_signal = convolve(continuous_signal, kernel, mode='same')
    return time_array, smoothed_signal

# Example usage with combined spike data
kernel_size = 100  # Length of the kernel in samples
sigma = 10  # Standard deviation of the Gaussian kernel
kernel = gaussian_kernel(kernel_size, sigma)

# Assuming `all_spike_times` contains combined spike times from all populations
all_spike_times = np.concatenate([tcr_spike_times, in_spike_times, trn_spike_times])  # Concatenate spike times
sampling_rate = 1000  # Hz
dt = 1.0 / sampling_rate  # Time step in seconds
total_time = 1000  # Total time of the experiment in milliseconds

time_array, combined_continuous = convert_spikes_to_continuous(all_spike_times, total_time, dt, kernel)

# Plotting the continuous signal
plt.figure(figsize=(12, 6))
plt.plot(time_array, combined_continuous, label='Combined Continuous Signal from All Populations')
plt.title('Continuous Signal from Combined Spike Data')
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


TypeError: only size-1 arrays can be converted to Python scalars

In [17]:
# Extracting spike times from the Segment objects
def extract_spike_times(segment):
    return [np.array(st.magnitude) for st in segment.spiketrains]

# Using the extraction function on your existing data
tcr_spike_times = extract_spike_times(TCR_spikes.segments[0])
in_spike_times = extract_spike_times(IN_spikes.segments[0])
trn_spike_times = extract_spike_times(TRN_spikes.segments[0])

In [19]:
def convert_spikes_to_continuous(spike_times, total_time, dt, kernel):
    # Create a timeline with zero amplitude
    continuous_signal = np.zeros(int(total_time / dt) + 1)
    time_array = np.linspace(0, total_time, num=len(continuous_signal))
    
    # Increment the continuous signal at the indices of each spike time
    indices = (spike_times / dt).astype(int)
    indices = indices[indices < len(continuous_signal)]  # Ensure indices are within bounds
    continuous_signal[indices] += 1
    
    # Convolve the spike signal with the kernel to smooth it
    smoothed_signal = convolve(continuous_signal, kernel, mode='same')
    return time_array, smoothed_signal

# Create Gaussian Kernel
kernel_size = 100  # in terms of number of samples
std_dev = 10  # standard deviation in samples
kernel = gaussian_kernel(kernel_size, std_dev)

# Parameters
sampling_rate = 1000  # Hz
dt = 1.0 / sampling_rate
total_time = 1000  # ms

# Generate continuous signal for combined spikes
combined_time, combined_continuous = convert_spikes_to_continuous(all_spike_times, total_time, dt, kernel)

# Plotting the continuous signal
plt.figure(figsize=(12, 6))
plt.plot(combined_time, combined_continuous, label='Combined Continuous Signal from All Populations')
plt.title('Continuous Signal from Combined Spike Data')
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

NameError: name 'gaussian' is not defined

In [20]:
def perform_fft_and_plot_psd(signal, sampling_rate, title):
    n = len(signal)
    fft_result = np.fft.fft(signal)
    frequencies = np.fft.fftfreq(n, 1 / sampling_rate)
    
    positive_frequencies = frequencies > 0
    psd = np.abs(fft_result[positive_frequencies]) ** 2 / (n * sampling_rate)
    
    plt.figure(figsize=(10, 5))
    plt.plot(frequencies[positive_frequencies], psd)
    plt.title('PSD of ' + title)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD [V²/Hz]')
    plt.xlim(0, 100)  # Up to 100 Hz
    plt.grid(True)
    plt.show()

# Perform FFT and plot PSD
perform_fft_and_plot_psd(combined_continuous, sampling_rate, 'PSD of Combined Continuous Signal')

NameError: name 'combined_continuous' is not defined